이는 특정 한 문장에 대한 결과를 확인할 수 있고, 레시피 1차 태깅을 할 수 있는 코드이다.\
특정 한 문장에 대한 결과만 확인하고 싶다면, 공통적 실행부분 + 문장 예측 결과 확인 코드만 실행하면 되고, 레시피 1차 태깅을 하고 싶다면, 모든 코드를 실행해야 한다.

# 공통적 실행 부분

In [ ]:
!pip install transformers seqeval[gpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizerFast, ElectraConfig, ElectraForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


구글 드라이브를 연동한다.\
파인튜닝한 모델과 토크나이저와 태깅할 데이터를 불러오기 위해 필요하다.\
따라서 사전에 구글 드라이브에 모델, 토크나이저, 각종 데이터를 업로드 해야 한다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# 문장 예측 결과 확인

최종 테스트 데이터셋을 불러온다.\
레이블 리스트를 구성하는데 사용된다.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/2022_lesik_workspace/lesik/data/new_corpus_recipe_test_dataset.tsv', sep = '\t', keep_default_na=False)
df.head()

,text,label
0,1. 깨끗이 씻은 우엉과 당근은 껍질을 얇게 벗긴 후 채칼이나 칼로 가늘게 채를 썰...,O O CV_STATE CV_STATE CV_STATE CV_INGREDIENT C...
1,(tip. 식촛물에 우엉을 1시간 정도 담가 두면 우엉의 아린 맛을 제거할 수 있어요),O O O O CV_SEASONING CV_SEASONING CV_SEASONING...
2,(채칼을 사용하여 채를 썰게 되면 더욱 간단하답니다),O O O O O O O O O O O O O O O O
3,2. 달군 팬에 오일을 약간 두른 후 당근에 소금을 살짝 뿌려 간을 해 볶아주세요.,O O O O O O CV_SEASONING O O O O CV_INGREDIENT...
4,"3. 달군 팬에 오일을 약간 두른 후 우엉의 숨이 죽을 때까지 볶다가, 우엉 조림 ...",O O O O O O CV_SEASONING O O O O CV_INGREDIENT...


In [ ]:
# Split labels based on whitespace and turn them into a list
arr_labels = set()
for lb in df.label:
    lb = lb.split()
    for ll in lb:
        if ll not in arr_labels:
            arr_labels.add(ll)
print(arr_labels)

unique_labels = {'OGG_EDUCATION', 'MT_ELEMENT', 'AFW_OTHER_PRODUCTS', 'MT_ROCK', 'TI_OTHERS', 'PS_NAME', 'CV_BUILDING_TYPE', 'AM_REPTILIA', 'OGG_FOOD', 'AF_MUSICAL_INSTRUMENT', 'AF_BUILDING', 'AFA_MUSIC', 'CV_SPORTS_INST', 'QT_ORDER', 'TM_COLOR', 'LCG_MOUNTAIN', 'QT_MAN_COUNT', 'PS_CHARACTER', 'AM_OTHERS', 'OGG_LIBRARY', 'TMM_DISEASE', 'OGG_MEDICINE', 'LCG_ISLAND', 'TI_MINUTE', 'MT_CHEMICAL', 'TM_CELL_TISSUE_ORGAN', 'QT_OTHERS', 'CV_TRIBE', 'QT_TEMPERATURE', 'PT_FLOWER', 'OGG_POLITICS', 'DT_WEEK', 'FD_ART', 'AM_AMPHIBIA', 'FD_MEDICINE', 'AF_CULTURAL_ASSET', 'AF_TRANSPORT', 'EV_SPORTS', 'LCG_CONTINENT', 'PT_TREE', 'TMI_SERVICE', 'AM_MAMMALIA', 'TM_SPORTS', 'CV_INGREDIENT', 'OGG_HOTEL', 'QT_PHONE', 'CV_LANGUAGE', 'CV_FUNDS', 'CV_CURRENCY', 'FD_OTHERS', 'LCG_RIVER', 'LCP_CAPITALCITY', 'LC_OTHERS', 'QT_SIZE', 'TM_CLIMATE', 'TM_SHAPE', 'CV_POLICY', 'EV_ACTIVITY', 'TR_ART', 'QT_ADDRESS', 'OGG_RELIGION', 'CV_POSITION', 'FD_HUMANITIES', 'CV_CULTURE', 'QT_SPORTS', 'QT_ALBUM', 'CV_ART', 'CV_FOOD', 'CV_LAW', 'OGG_MILITARY', 'DT_DAY', 'FD_SOCIAL_SCIENCE', 'LCP_PROVINCE', 'CV_CLOTHING', 'TI_HOUR', 'DT_DYNASTY', 'DT_SEASON', 'FD_SCIENCE', 'TMI_HW', 'OGG_SPORTS', 'TR_OTHERS', 'TM_DIRECTION', 'TMI_SITE', 'QT_LENGTH', 'MT_METAL', 'LCG_OCEAN', 'DT_OTHERS', 'LCP_COUNTY', 'TMIG_GENRE', 'OGG_ECONOMY', 'TMI_SW', 'CV_SPORTS_POSITION', 'AFA_DOCUMENT', 'PT_OTHERS', 'AFA_ART_CRAFT', 'EV_OTHERS', 'TMI_EMAIL', 'QT_PRICE', 'EV_FESTIVAL', 'TI_SECOND', 'CV_TAX', 'O', 'QT_VOLUME', 'AF_WEAPON', 'LCG_BAY', 'OGG_SCIENCE', 'PT_FRUIT', 'CV_OCCUPATION', 'QT_CHANNEL', 'OGG_ART', 'AM_INSECT', 'CV_FOOD_STYLE', 'QT_PERCENTAGE', 'OGG_LAW', 'TR_SCIENCE', 'CV_RELATION', 'AM_PART', 'QT_AGE', 'TMI_MODEL', 'AM_BIRD', 'OGG_OTHERS', 'CV_SPORTS', 'DT_YEAR', 'LCP_COUNTRY', 'AFA_VIDEO', 'DT_GEOAGE', 'TI_DURATION', 'AM_TYPE', 'CV_SEASONING', 'AM_FISH', 'CV_PRIZE', 'PS_PET', 'AFW_SERVICE_PRODUCTS', 'TMI_PROJECT', 'CV_DRINK', 'LC_SPACE', 'LCP_CITY', 'EV_WAR_REVOLUTION', 'AFA_PERFORMANCE', 'QT_SPEED', 'PT_GRASS', 'DT_MONTH', 'PT_PART', 'OGG_MEDIA', 'PT_TYPE', 'TMM_DRUG', 'AF_ROAD', 'DT_DURATION', 'TR_MEDICINE', 'TR_HUMANITIES'}
print(unique_labels)

# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

prob_tag_list=[]
for lb in arr_labels:
    if lb not in labels_to_ids:
        prob_tag_list.append("'"+lb+"'")
print(",".join(prob_tag_list))


labels_to_ids['CV_ACT'] = 150
ids_to_labels[150] = 'CV_ACT'
labels_to_ids['CV_STATE'] = 151
ids_to_labels[151] = 'CV_STATE'
print(labels_to_ids)
print(ids_to_labels)

{'QT_VOLUME', 'QT_TEMPERATURE', 'CV_SEASONING', 'CV_INGREDIENT', 'TI_DURATION', 'CV_STATE', 'O'}
{'TMI_SITE', 'FD_ART', 'CV_CURRENCY', 'CV_DRINK', 'OGG_LIBRARY', 'LCG_ISLAND', 'PT_PART', 'PT_FRUIT', 'QT_MAN_COUNT', 'CV_TAX', 'OGG_SPORTS', 'AM_OTHERS', 'AM_AMPHIBIA', 'CV_TRIBE', 'OGG_MEDICINE', 'LCP_COUNTRY', 'DT_WEEK', 'TM_CELL_TISSUE_ORGAN', 'LCP_CITY', 'TM_CLIMATE', 'TM_DIRECTION', 'AFW_OTHER_PRODUCTS', 'LCP_CAPITALCITY', 'AFA_VIDEO', 'AM_MAMMALIA', 'LC_SPACE', 'QT_CHANNEL', 'AF_ROAD', 'EV_SPORTS', 'AM_TYPE', 'TMI_MODEL', 'AF_MUSICAL_INSTRUMENT', 'EV_OTHERS', 'LCG_OCEAN', 'QT_SIZE', 'LCG_RIVER', 'O', 'EV_WAR_REVOLUTION', 'TI_SECOND', 'CV_INGREDIENT', 'LCG_CONTINENT', 'CV_CULTURE', 'PT_FLOWER', 'DT_DYNASTY', 'TMI_PROJECT', 'OGG_MEDIA', 'OGG_RELIGION', 'DT_DAY', 'CV_SPORTS_INST', 'TI_MINUTE', 'LCP_PROVINCE', 'QT_OTHERS', 'AM_BIRD', 'CV_POLICY', 'OGG_SCIENCE', 'QT_AGE', 'CV_BUILDING_TYPE', 'DT_DURATION', 'TI_HOUR', 'FD_OTHERS', 'FD_SCIENCE', 'TMM_DRUG', 'MT_METAL', 'OGG_FOOD', 'TI_OTHER

테스트 및 태깅하기를 원하는 모델과 토크나이저를 불러오는 함수이다.

In [ ]:
def load(epoch):
    model_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/model/POOLYOALNAM_CORPUS_STATE_EPOCH_'+ str(epoch)
    model = ElectraForTokenClassification.from_pretrained(model_directory, num_labels=len(labels_to_ids))


    model.to(device)
    
    tokenizer_directory = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/tokenizer/POOLYOALNAM_CORPUS_STATE_EPOCH_' +str(epoch)
    tokenizer = ElectraTokenizerFast.from_pretrained(tokenizer_directory)
    
    return model, tokenizer

In [ ]:
from transformers import ElectraTokenizerFast

MAX_LEN = 256
TEST_BATCH_SIZE = 8
EPOCH = 72
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
model, tokenizer = load(72)

In [ ]:
class ElectraDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index].strip()
        word_labels = self.data.label[index].split()

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        valid_token_list = []
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] == 0:
                continue
            valid_token_list.append(mapping)
        if len(valid_token_list) != len(word_labels):
            print(index, len(word_labels), len(valid_token_list), sentence)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        if len(labels) != 0:
            for idx, mapping in enumerate(encoding["offset_mapping"]):
                if mapping[0] == 0 and mapping[1] == 0:
                    continue
                tok = tokenizer.convert_ids_to_tokens(encoding['input_ids'][idx])
            
                # overwrite label
                if i == len(labels):
                    break
                encoded_labels[idx] = labels[i]
                i += 1
                
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['label'] = torch.as_tensor(encoded_labels)
        
        return item
    def __len__(self):
        return self.len

테스트 및 태깅하기를 원하는 모델과 토크나이저를 불러온다.\
argument로 불러오기를 원하는 epoch를 적는다.

In [ ]:
model, tokenizer = load(72)

모델이 문장을 예측하는 함수이다.

In [ ]:
def predict(sentence):
    result = []
    inputs = tokenizer(sentence,
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=MAX_LEN,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model.config.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    word_prediction = []
    token_prediction = []
    pred_dict = {}
    token_dict = {}
    for token_pred, word_idx in zip(wp_preds, inputs.word_ids()):
        if token_pred[0] not in ['[CLS]','[UNK]','[PAD]','[SEP]']:
            result.append([word_idx, token_pred[0], token_pred[1]]) # 
            #print(token_pred)
            if word_idx not in pred_dict:
                pred_dict[word_idx] = set()
            if word_idx not in token_dict:
                token_dict[word_idx] = ""
            pred_dict[word_idx].add(token_pred[1])
            token_dict[word_idx] += token_pred[0].replace("#", "")
                
    for token_pred, word_idx in zip(wp_preds, inputs.word_ids()):
        #only predictions on first word pieces are important
        if token_pred[0] not in ['[CLS]','[UNK]','[PAD]','[SEP]']:
            if token_pred[1] != 'O':
                token_prediction.append([word_idx, token_pred[0],token_pred[1]])
    
    #for i in range(len(token_prediction)):
    #    print(token_prediction[i], word_idx)
    #print()
    return result

테스트하고 싶은 문장을 predict 함수의 argument로 보낸다.\
아래 코드를 실행하면 토큰별 예측 결과가 나온다.\
**입력 예시**


> 잘게 다진 면을 갈아주세요.



**출력 예시**

> [[0, '잘', 'CV_STATE'],\
 [0, '##게', 'CV_STATE'],\
 [1, '다진', 'CV_STATE'],\
 [2, '면', 'CV_INGREDIENT'],\
 [2, '##을', 'O'],\
 [3, '갈아', 'O'],\
 [3, '##주', 'O'],\
 [3, '##세요', 'O'],\
 [4, '.', 'O']]



In [ ]:
predict("잘게 다진 면을 갈아주세요.")

[[0, '잘', 'CV_STATE'],
 [0, '##게', 'CV_STATE'],
 [1, '다진', 'CV_STATE'],
 [2, '면', 'CV_INGREDIENT'],
 [2, '##을', 'O'],
 [3, '갈아', 'O'],
 [3, '##주', 'O'],
 [3, '##세요', 'O'],
 [4, '.', 'O']]

# 레시피 1차 태깅

dir_path는 레이블링할 데이터가 들어있는 폴더 경로이다.\
wf는 레이블링한 결과가 저장될 파일 경로이다.

In [ ]:
# 태그 단위로 병합
import os
def preprocess():
  dir_path = '/content/gdrive/MyDrive/2022_lesik_workspace/lesik/data/recipe/10000/10000_kimjju'
  wf = open('/content/gdrive/MyDrive/2022_lesik_workspace/lesik/data/recipe/10000/labeled_kimjju.txt', 'w') # 쓸 파일
  for path in os.listdir(dir_path):
      if os.path.isfile(os.path.join(dir_path, path)):
        f = open(os.path.join(dir_path, path), 'r') # 읽을 파일 오픈
        recipe = f.readlines()
        f.close()
        #recipe = recipe[recipe.index("[조리방법]\n") + 1:]
        recipe_len = len(recipe)
        for i in range(recipe_len):
          if recipe[i] == '\n': recipe[i] = ""
        #print(recipe)
        recipe = list(filter(None, recipe)) # 레시피 조리방법 출력
        #print(recipe)

        result = []
        for r_part in recipe:
          pre_list = predict(r_part)
          #print("before", pre_list)
          length = len(pre_list)
          
          # 1. 같은 워드 같은 태그 가진 애 병합
          
          for idx in range(length-1):
            if pre_list[idx][0] == pre_list[idx+1][0] and pre_list[idx][2] == pre_list[idx+1][2]:
              pre_list[idx+1] = [pre_list[idx][0], pre_list[idx][1].replace("#", "") + pre_list[idx+1][1].replace("#", ""), pre_list[idx][2]]
              pre_list[idx] = []
            else: pre_list[idx] = pre_list[idx] = [pre_list[idx][0], pre_list[idx][1].replace("#", ""), pre_list[idx][2]]
          pre_list = list(filter(None, pre_list))
          #print("after", pre_list)

          # 문장 형성 및, 같은 워드아이디 가진 애 병합
          result = ""
          r_part_l = ""
          r_part_r = r_part
          #print(r_part)
          for part in pre_list:
            if r_part_r.find(part[1]) != -1 and part[2] != 'O':
              index = r_part_r.find(part[1])
              r_part_l = r_part_l + r_part_r[:index] + "<" + part[1] + ":" + part[2] + ">"
              r_part_r = r_part_r[index + len(part[1]):]
              #print("r_part_l", r_part_l)
              #print("r_part_r", r_part_r)
          result = r_part_l + r_part_r
          wf.write(result)
            
  wf.close()

아래 코드를 실행하면, 레이블링이 된다.\
예를 들어, 아래의 코드를 실행하면, "당근즙, 시금치즙을 우유와 섞어주세요."와 같은 문장은\
"<당근즙:CV_SEASONING>, <시금치즙:CV_SEASONING>을 <우유:CV_SEASONING>와 섞어주세요."와 같이 태깅된다.

In [ ]:
preprocess()